In [ ]:
from questions import data
from textbook_q import *
from chapter2 import chapter
import os
import json
import threading
import shutil
import argparse
import openai
from tqdm import tqdm
import time
import pickle
from nltk import sent_tokenize
import random
import string
from tqdm import tqdm

In [ ]:
def gpt_new_conversation(prompt, temperature=0, pp=0):
    """
    Input: The prompt temperature (default 0) and presence penalty (default 0)
    Output: A list of dict that contains the user prompt and ChatGPT's answer to the prompt
    """

    try:
        conversation = [{'role': 'user', 'content': prompt}]
        response = openai.ChatCompletion.create(
            model="gpt-4-0314",
            messages=conversation,
            temperature=temperature,
            request_timeout=120,
            presence_penalty=pp,
            max_tokens=2000,
            top_p=1,
            frequency_penalty=0)
    #     print(response)
        content = response.choices[0]['message']['content']
        # Remove the first sentence of the answer
        role = response.choices[0]['message']['role']
        conversation.append({'role': role, 'content': content})
        # This can be useful to calculate the cost
        # cost = response.usage['total_tokens']

        return conversation
    except:
        print("*** ChatGPT has time out. If you see this message too many times, please consider stop ***")
        time.sleep(5)
        return gpt_new_conversation(prompt, temperature)


In [ ]:
def generate_sample_review_questions(topic, question, answer, sub_chapter):
    prompt = f"""You are the instructor for a college introductory level nano engineering course \
    designed for students with only rudimentary understandings of physics and chemistry. Right now you are \
    doing a review session for students to prepare for their finals on the topic of {topic}. You need to \
    come up with an original review question and correct answer that closely follow the style of a sample\
    review question and answer written by a real instructor for this course. Your review question and answer \
    should base on the contents of a textbook chapter on the topic of {topic}. Below you will see the chapter and also \
    the sample review question and answer whose style you should follow.\
    
    Sample review question : {question}
    
    Sample answer for the question: {answer}
    
    Textbook chapter : {sub_chapter}
    
    The output format is *important*, you need to return your generated review question and answer in the format of a \
    *python dictionary*,  "question": your generated review question, "answer": your generated answer. 
    """
    convo = gpt_new_conversation(prompt, temperature = 0.2)
    
    return convo
    

In [ ]:
def remove_trail_punct(text):
    i=0
    
    for t in text:
        if t.lower().isalpha():
            start = i
            break
        else:
            i+=1
    i =len(text)-1
    for t in text[::-1]:
        if t.lower().isalpha():
            end = i
            break
        else:
            i-=1
    return  text[start:end+1]
        

In [ ]:
ret_list =[]
for count in tqdm(range(50)):

    x = random.choice(list(range(len(chapter))))
    y = random.choice(list(range(len(textbook_questions))))
    sub_chapter = chapter[x]['text']
    topic = chapter[x]['topic']

    question = textbook_questions[y]
    answer = textbook_answers[y]
    print(topic)
    convo = generate_sample_review_questions(topic = topic, question =question, answer = answer, sub_chapter= sub_chapter)
    
    ret = {"topic":topic, "sub_chapter":sub_chapter, "sample_question": question, "sample_answer": answer, "convo": convo}
    ret_list.append(ret)
    with open("gpt_review_questions.pickle", "wb") as f:
        pickle.dump(ret_list, f)
    shutil.copyfile("gpt_review_questions.pickle", "gpt_review_questions_copy.pickle")
    
    
    

In [ ]:
for i in range(len(ret_list)):
    convo = ret_list[i]
    a = convo['convo'][1]['content'].replace(""""question":""", "QUESTION")
    a = a.replace(""""answer":""", "ANSWER")
    b= a.split("QUESTION")
    c = b[1].split('ANSWER')
    
    question = remove_trail_punct(c[0])
    answer = remove_trail_punct(c[1])
    review = {"question": question, "answer": answer}
    ret_list[i]["review"] = review

In [ ]:
with open("review_questions_50.pickle", "wb") as f:
    pickle.dump(ret_list, f)